<a href="https://colab.research.google.com/github/LucianaNieto/Cambodia_2023/blob/main/1_Cambodia_ChangeDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
# Import necessary packages
!pip install geemap -q

import pandas as pd
import geemap
import ee

# Initialize the Earth Engine API
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=ViD56ZnwjC7OTy4PYeAXcRjwqLsfPw7wLKN9w1eNF84&tc=yyu-DM97AtRa4rdF7OajzUL0pp27JcKEqT2zEb7Dzgs&cc=fY4-jx56KE3z5NG0Vx8z-xGQH_bqW1FptOzgWrsnLIc

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh4FDeEnUJQlQ1h2rPT6lK85U7Sgk8siUFZyyrTOGlJ0rKTCf8a3MXg

Successfully saved authorization token.


In [59]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
point = ee.Geometry.Point([103.37259922076987,13.151996991897148]) #Battamband Cambodia 

#open and merge both collections since they are in different folders 
c1 = ee.ImageCollection('projects/phenology-052020/assets/cambodia_2020_13E') #collection with 2020 data
c2 = ee.ImageCollection('users/luzluz2054/cambodia_2021_13E') #collection with 2021 data 
collectionMerged = c1.merge(c2) #merge both imagecollections 


In [60]:
col = collectionMerged.filterBounds(point) #filter appropiate features

In [61]:
#map different indices over the merged collection 

col = col.map(lambda image: image
              .addBands(image.expression(
                  '(NIR - RED) / (NIR + RED)', {
                      'NIR': image.select('B4'),
                      'RED': image.select('B3')
                      }).rename('NDVI'))
              .addBands(image.expression(
                  '(GREEN - NIR) / (GREEN + NIR)', {
                      'NIR': image.select('B4'),
                      'GREEN': image.select('B2')
                      }).rename('NDWI'))              
              .addBands(image.expression(
                  '(1 + L)*(NIR - RED) / (NIR + RED+ L)', {
                      'NIR': image.select('B4'),
                      'RED': image.select('B3'),
                      'L': 0.5 }).rename('SAVI')))


In [ ]:
vis_params = {
    'min': -1,
    'max': 1,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

# Add Earth Engine layers to Map
Map.addLayer(
    col,
    {'bands': ['NDVI'], 'min': -1, 'max': 1, 'palette':['a35c74','e8c714','005555']},
    'NDVI',
)
Map.addLayer(
    col,
    {'bands': ['SAVI'], 'min': -1, 'max': 1, 'palette':['faebd7','efbd7d','e69936','995e13','5e3a0c']},
    'SAVI',
)
Map.centerObject(point, 12)

#Map

In [62]:

nochange = ee.FeatureCollection('users/luzluz2054/NoChange_cambodia')
change = ee.FeatureCollection('users/luzluz2054/Change_cambodia')

rgb_vis = {'min': 0.0,'max': 3000,'bands': ['B3', 'B2', 'B1']}


filtered = col.filter(ee.Filter.date('2020-01-01', '2020-01-31')) 
image_0102 = filtered.median()

# Display the input composite.
#Map.addLayer(image_0102, rgb_vis, '2020_01_02')

filtered = col.filter(ee.Filter.date('2020-02-01', '2020-03-01')) \
             #.filter(ee.Filter.bounds(agriculture)) \
             #.map(mask_s2clouds)

image_0203 = filtered.median()
#Map.addLayer(image_0203, rgb_vis, '2020_02_03')

stacked_image = image_0102.addBands(image_0203)

# Overlay the point on the image to get training data.
training = stacked_image.sampleRegions(
    collection=change.merge(nochange), 
    properties= ['change'], 
    scale=3
)

# Train a classifier.
classifier = ee.Classifier.smileRandomForest(100).train(
    features=training,  
    classProperty='change', 
    inputProperties=stacked_image.bandNames()
)

# Classify the image.
classified = stacked_image.classify(classifier)
Map.addLayer(classified, {'min': 0, 'max': 1, 'palette': ['white', '#426270']}, 'change')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Spectral Distance

In [63]:
agriculture = ee.FeatureCollection('users/luzluz2054/agriculture')
Map.addLayer(agriculture)
# Define the RGB visualization parameters
rgb_vis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B3', 'B2', 'B1'],
}


filtered = col.filter(ee.Filter.bounds(agriculture)).select('B1', 'B2', 'B3','B4', 'NDVI', 'SAVI', 'NDWI')
# Define the date of incident
date_of_incident = ee.Date('2020-03-20')

# Create image collections for before and after the incident

before = filtered.filterDate(date_of_incident.advance(-2, 'month'), date_of_incident).median().clip(agriculture)
after = filtered.filterDate(date_of_incident, date_of_incident.advance(2, 'month')).median().clip(agriculture)

# Add the before and after layers to the map
Map.addLayer(before, rgb_vis, 'Before')
Map.addLayer(after, rgb_vis, 'After')

# Compute the spectral angle
angle = after.spectralDistance(before, 'sam')
Map.addLayer(angle, {'min': 0, 'max': 1, 'palette': ['white', 'purple']}, 'Spectral Angle')

# Compute the squared Euclidean distance
sed = after.spectralDistance(before, 'sed')

# Take the square root to get the Euclidean distance
distance = sed.sqrt()

# Add the spectral distance layer to the map
Map.addLayer(distance, {'min': 0, 'max': 1500, 'palette': ['white', '#81a19c']}, 'spectral distance')
Map

Map(bottom=754.0, center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [86]:
dates = [ee.Date('2020-01-05'),ee.Date('2020-01-10'),ee.Date('2020-01-15'),ee.Date('2020-01-20'),ee.Date('2020-01-25'),ee.Date('2020-01-30'),
         ee.Date('2020-02-05'),ee.Date('2020-02-10'),ee.Date('2020-02-15'),ee.Date('2020-02-20'),ee.Date('2020-02-25'),ee.Date('2020-02-28'),
         ee.Date('2020-03-05'),ee.Date('2020-03-10'),ee.Date('2020-03-15'),ee.Date('2020-03-20'),ee.Date('2020-03-25'),ee.Date('2020-03-30'),
         ee.Date('2020-04-05'),ee.Date('2020-04-10'),ee.Date('2020-04-15'),ee.Date('2020-04-20'),ee.Date('2020-04-25'),ee.Date('2020-04-28'),
         ee.Date('2021-01-05'),ee.Date('2021-01-10'),ee.Date('2021-01-15'),ee.Date('2021-01-20'),ee.Date('2021-01-25'),ee.Date('2021-01-30'),
         ee.Date('2021-02-05'),ee.Date('2021-02-10'),ee.Date('2021-02-15'),ee.Date('2021-02-20'),ee.Date('2021-02-25'),ee.Date('2021-02-28'),
         ee.Date('2021-03-05'),ee.Date('2021-03-10'),ee.Date('2021-03-15'),ee.Date('2021-03-20'),ee.Date('2021-03-25'),ee.Date('2021-03-30'),
         ee.Date('2021-04-05'),ee.Date('2021-04-10'),ee.Date('2021-04-15'),ee.Date('2021-04-20'),ee.Date('2021-04-25'),ee.Date('2021-04-30')
]

In [90]:
# Get the geometry of the "agriculture" object
agriculture = ee.FeatureCollection('users/luzluz2054/agriculture')

# Define a list to store the before and after date combinations and distances
date_distances = []

# Loop over the date range and find the combinations and distances
for date in dates: 
    # Create image collections for before and after the incident
    before = filtered.filterDate(date.advance(-2, 'day'), date).median().clip(agriculture)
    after = filtered.filterDate(date, date.advance(2, 'day')).median().clip(agriculture)

    # Compute the squared Euclidean distance
    distance = after.spectralDistance(before, 'sed')

    # Take the square root to get the Euclidean distance
    distance = distance.sqrt()

    # Calculate the mean distance within the "agriculture" object
    mean_distance = distance.reduceRegion(reducer=ee.Reducer.mean(), geometry=agriculture, scale=10).get('distance')

    # Store the before and after date combinations and distances in a dictionary
    date_distance = {
        'before_date': date.advance(-2, 'day').format('YYYY-MM-dd').getInfo(),
        'after_date': date.advance(2, 'day').format('YYYY-MM-dd').getInfo(),
        'distance': mean_distance.getInfo()
    }

    # Append the dictionary to the list
    date_distances.append(date_distance)

# Sort the list by distance in descending order
sorted_date_distances = sorted(date_distances, key=lambda x: x['distance'], reverse=True)

# Print the sorted list
sorted_date_distances


[{'before_date': '2021-03-18',
  'after_date': '2021-03-22',
  'distance': 118.04234768294235},
 {'before_date': '2021-04-18',
  'after_date': '2021-04-22',
  'distance': 112.71331960044223},
 {'before_date': '2020-02-13',
  'after_date': '2020-02-17',
  'distance': 111.48604241135101},
 {'before_date': '2020-02-18',
  'after_date': '2020-02-22',
  'distance': 103.8708109415034},
 {'before_date': '2020-04-26',
  'after_date': '2020-04-30',
  'distance': 101.12686945756707},
 {'before_date': '2021-02-13',
  'after_date': '2021-02-17',
  'distance': 92.90680741521923},
 {'before_date': '2020-02-23',
  'after_date': '2020-02-27',
  'distance': 88.99249926527949},
 {'before_date': '2020-01-28',
  'after_date': '2020-02-01',
  'distance': 85.76314081130205},
 {'before_date': '2021-03-28',
  'after_date': '2021-04-01',
  'distance': 79.84079231876628},
 {'before_date': '2021-01-23',
  'after_date': '2021-01-27',
  'distance': 79.49477471250731},
 {'before_date': '2021-02-18',
  'after_date':

In [85]:
date_changed_1 = col.filter(ee.Filter.bounds(agriculture)).select('B1', 'B2', 'B3','B4', 'NDVI', 'SAVI', 'NDWI').filterDate('2020-02-08', '2020-02-09')
date_changed_2 = col.filter(ee.Filter.bounds(agriculture)).select('B1', 'B2', 'B3','B4', 'NDVI', 'SAVI', 'NDWI').filterDate('2020-02-12', '2020-02-13')
Map.addLayer(date_changed_1, rgb_vis, 'D1')
Map.addLayer(date_changed_2, rgb_vis, 'D2')

Map

Map(bottom=485956.0, center=[13.146352385470161, 103.40778350830078], controls=(WidgetControl(options=['positi…